# Basic Search

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---
                                                                    
Before we do any work, we needs to import these functions cdapython.
We're also telling cdapython to report it's version so we can be sure we're using the one we mean to:

In [1]:
from cdapython import Q, columns, unique_terms, query
print(Q.get_version())

2022.6.28


In [2]:
Q.set_default_project_dataset("broad-dsde-dev.cda_dev")
Q.set_host_url("https://cancerdata.dsde-dev.broadinstitute.org/")
Q.get_host_url()
Q.get_default_project_dataset()

'broad-dsde-dev.cda_dev'

<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">
    
CDA data comes from three sources:
<ul>
<li><b>The <a href="https://proteomic.datacommons.cancer.gov/pdc/"> Proteomic Data Commons</a> (PDC)</b></li>
<li><b>The <a href="https://gdc.cancer.gov/">Genomic Data Commons</a> (GDC)</b></li>
<li><b>The <a href="https://datacommons.cancer.gov/repository/imaging-data-commons">Imaging Data Commons</a> (IDC)</b></li>
</ul> 
    
The CDA makes this data searchable in four main endpoints:

<ul>
<li><b>subject:</b> A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</li>
<li><b>researchsubject:</b> A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. An individual who participates in 3 studies will have 3 researchsubject IDs</li>
<li><b>specimen:</b> Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</li>
<li><b>file:</b> A unit of data about subjects, researchsubjects, specimens, or their associated information</li>
</ul>
and two endpoints that offer deeper information about data in the researchsubject endpoint:
<ul>
<li><b>diagnosis:</b> A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</li>
<li><b>treatment:</b> Represent medication administration or other treatment types.</li>
</ul>
Any metadata field can be searched from any endpoint, the only difference between search types is what type of data is returned by default. This means that you can think of the CDA as a really, really enormous spreadsheet full of data. To search this enormous spreadsheet, you'd want select columns, and then filter rows.
</div>


If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specfic format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, start with `file`.

In CDA search, these concepts can also be chained together, so you can look specifically for specimen subjects, or researchsubject diagnoses. In the four 'main' tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in specimen files. Diagnosis and treatment do not have files directly associated with them and so can only be used to find files in conjunction with the other searches.

In all cases, any search can use any metadata field, the only difference between search types is what type of data you return by default. 



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` and save it to a variable `myquery`:

In [3]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')


<div class="cdawarn" style="background-color:#f9cfbf;color:black;padding:20px;">
    
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what Subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [4]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 3786 ms


We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [5]:
subjectresults


            QueryID: 60d928ef-0741-4f38-8f2f-941c02d84fec
            
            Offset: 0
            Count: 100
            Total Row Count: 2384
            More pages: True
            

This output tells us our QueryID, which we don't really need, but the computer does to track our questions. Then it tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---
    
We have 2384 subject matches, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [6]:
subjectresults.to_dataframe()

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,days_to_death,cause_of_death
0,ACRIN-DSC-MR-Brain-048,"[{'system': 'IDC', 'value': 'ACRIN-DSC-MR-Brai...",Homo sapiens,None,None,None,NaN,[acrin_dsc_mr_brain],None,NaN,None
1,ACRIN-FMISO-Brain-050,"[{'system': 'IDC', 'value': 'ACRIN-FMISO-Brain...",Homo sapiens,None,None,None,NaN,[acrin_fmiso_brain],None,NaN,None
2,C3L-04795,"[{'system': 'IDC', 'value': 'C3L-04795'}]",Homo sapiens,None,None,None,NaN,[cptac_gbm],None,NaN,None
3,C3N-02186,"[{'system': 'GDC', 'value': 'C3N-02186'}, {'sy...",homo sapiens,male,not reported,not reported,-19526.0,"[CPTAC-3, cptac_gbm, CPTAC3-Discovery]",Alive,NaN,None
4,C3N-03186,"[{'system': 'GDC', 'value': 'C3N-03186'}, {'sy...",homo sapiens,female,not reported,not reported,-20009.0,"[CPTAC-3, cptac_gbm, CPTAC3-Discovery]",Dead,646.0,Cancer Related
...,...,...,...,...,...,...,...,...,...,...,...
95,QIN-GBM-TR-44,"[{'system': 'IDC', 'value': 'QIN-GBM-TR-44'}]",Homo sapiens,None,None,None,NaN,[qin_gbm_treatment_response],None,NaN,None
96,TCGA-02-0027,"[{'system': 'GDC', 'value': 'TCGA-02-0027'}, {...",homo sapiens,female,white,not hispanic or latino,-12369.0,"[tcga_gbm, TCGA-GBM]",Dead,370.0,None
97,TCGA-06-0747,"[{'system': 'GDC', 'value': 'TCGA-06-0747'}, {...",homo sapiens,male,white,not hispanic or latino,-19657.0,"[tcga_gbm, TCGA-GBM]",Dead,82.0,None
98,TCGA-06-0877,"[{'system': 'GDC', 'value': 'TCGA-06-0877'}, {...",homo sapiens,male,white,not hispanic or latino,-28760.0,"[tcga_gbm, TCGA-GBM]",Dead,NaN,None


By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any pandas functions to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs:


In [7]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", na=False)]

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,days_to_death,cause_of_death
69,TCGA-08-0521,"[{'system': 'GDC', 'value': 'TCGA-08-0521'}, {...",homo sapiens,male,black or african american,hispanic or latino,-6377.0,"[tcga_gbm, TCGA-GBM]",Dead,146.0,None


There are subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. 

We'll cover how to work with large results dataframes in the <a href="../Pagination">Pagination</a> notebook. Or, learn how to get summary information from search results in the <a href="../DataSummaries">Data Summaries</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</i>

    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.",STRING</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>species:</b> The taxonomic group (e.g. species) of the subject. For MVP, since taxonomy vocabulary is consistent between GDC and PDC, using text. Ultimately, this will be a term returned by the vocabulary service.</li>
<li><b>sex:</b> The biologic character or quality that distinguishes male and female from one another as expressed by analysis of the person's gonadal, morphologic (internal and external), chromosomal, and hormonal characteristics.</li>
<li><b>race:</b> An arbitrary classification of a taxonomic group that is a division of a species. It usually arises as a consequence of geographical isolation within a species and is characterized by shared heredity, physical attributes and behavior, and in the case of humans, by common history, nationality, or geographic distribution. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>ethnicity:</b> An individual's self-described social and cultural grouping, specifically whether an individual describes themselves as Hispanic or Latino. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days.</li>
<li><b>subject_associated_project:</b> The list of Projects associated with the Subject.</li>
<li><b>vital_status:</b> Coded value indicating the state or condition of being living or deceased; also includes the case where the vital status is unknown.</li>
<li><b>days_to_death:</b> Number of days between the date used for index and the date from a person's date of death represented as a calculated number of days.</li>
<li><b>cause_of_death:</b> Coded value indicating the circumstance or condition that results in the death of the subject.</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our critera, we can also run our query against the researchsubject endpoint:

In [8]:
researchsubjectresults = myquery.researchsubject.run()
researchsubjectresults

Getting results from database

Total execution time: 3968 ms



            QueryID: bd1c1b69-827e-4b7a-a879-06627ce21d88
            
            Offset: 0
            Count: 100
            Total Row Count: 3716
            More pages: True
            

Now we see that our 2384 subjects have 3716 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [9]:
researchsubjectresults.to_dataframe()

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,9c32418f-ce23-4259-86f8-0b3b46fcfe33,"[{'system': 'GDC', 'value': '9c32418f-ce23-425...",GENIE-JHU,"Neoplasms, NOS",Brain,GENIE-JHU-03295
1,b6e6624f-5776-4d7e-a2c1-595b2b539f46,"[{'system': 'GDC', 'value': 'b6e6624f-5776-4d7...",GENIE-JHU,"Malignant Lymphomas, NOS or Diffuse",Brain,GENIE-JHU-02253
2,51343e16-05b6-4314-8169-59ff765bda0a,"[{'system': 'GDC', 'value': '51343e16-05b6-431...",EXCEPTIONAL_RESPONDERS-ER,"Neoplasms, NOS",Brain,ER-B0DO
3,54ada9c7-e8f7-4ecc-b829-903be7e3ad82,"[{'system': 'GDC', 'value': '54ada9c7-e8f7-4ec...",GENIE-NKI,"Neoplasms, NOS",Brain,GENIE-NKI-8IN3
4,be68cfb0-c47f-49a1-9e39-b0b3e57372fc,"[{'system': 'GDC', 'value': 'be68cfb0-c47f-49a...",GENIE-JHU,"Neoplasms, NOS",Brain,GENIE-JHU-02615
...,...,...,...,...,...,...
95,5838a973-c162-47cd-a93d-e28fd92f1ca5,"[{'system': 'GDC', 'value': '5838a973-c162-47c...",CPTAC-3,Gliomas,Brain,C3L-01046
96,ba037761-7166-40e1-8f27-1c1561a18b9c,"[{'system': 'GDC', 'value': 'ba037761-7166-40e...",CPTAC-3,Gliomas,Brain,C3L-02970
97,c5d83fdb-5a16-44c2-b392-8deb1a3c5b57,"[{'system': 'GDC', 'value': 'c5d83fdb-5a16-44c...",CPTAC-3,Gliomas,Brain,C3L-01327
98,5718a50d-332c-4882-8736-2ca8989946dd,"[{'system': 'GDC', 'value': '5718a50d-332c-488...",CPTAC-3,Gliomas,Brain,C3L-02984


Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered near the end of the <a href="../BuildingACohort/#merging-results-across-endpoints">Cohort Building workflow</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. An individual who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. For CDA, this is case_id.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>member_of_research_project:</b> A reference to the Study(s) of which this ResearchSubject is a member.</li>
<li><b>primary_diagnosis_condition:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>primary_diagnosis_site:</b> The text term used to describe the primary site of disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This categorization groups cases into general categories. This attribute represents the primary site of disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>subject_id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about researchsubjects that have a diagnosis that meets our search criteria:

In [16]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults #1756

Getting results from database

Total execution time: 4270 ms



            QueryID: a0b30464-8dc1-4025-94d6-a57183878bc4
            
            Offset: 0
            Count: 100
            Total Row Count: 1756
            More pages: True
            

In [10]:

diagnosisresults.to_dataframe()

Getting results from database

Total execution time: 3845 ms


,id,identifier,primary_diagnosis,age_at_diagnosis,morphology,stage,grade,method_of_diagnosis,subject_id,researchsubject_id
0,0607a306-8b24-5f44-bf0d-fbfcc9985b8a,"[{'system': 'GDC', 'value': '0607a306-8b24-5f4...",Glioblastoma,18383.0,9440/3,None,not reported,None,TCGA-08-0358,ec504558-e96a-44e9-835a-7b12680681ed
1,3c999848-2848-11ec-b712-0a4e2186f121,"[{'system': 'PDC', 'value': '3c999848-2848-11e...",Glioblastoma,19879.0,9440/3,Not Reported,Not Reported,,C3L-02704,104c2847-2139-11ea-aee1-0e1aae319e49
2,65dfa780-27c2-53e8-8112-df7ef3f6cf13,"[{'system': 'GDC', 'value': '65dfa780-27c2-53e...","Oligodendroglioma, anaplastic",19149.0,9451/3,None,not reported,None,TCGA-QH-A6CU,76cb16f9-37be-450e-b60a-5f0d0f1ec2bc
3,7336271c-f7db-5bd3-a590-7b7e3340bf9c,"[{'system': 'GDC', 'value': '7336271c-f7db-5bd...","Oligodendroglioma, NOS",13968.0,9450/3,None,not reported,None,TCGA-HT-7472,482d4078-7576-48cf-bf7c-29014935fd47
4,7eacc35e-fe59-58d6-936a-a92d72fb4f2b,"[{'system': 'GDC', 'value': '7eacc35e-fe59-58d...","Astrocytoma, anaplastic",17692.0,9401/3,None,not reported,None,TCGA-QH-A6XC,62088274-a45b-4616-b086-b0ddd64895f0
...,...,...,...,...,...,...,...,...,...,...
95,d091e91b-ff5e-11e9-9a07-0a80fada099c,"[{'system': 'PDC', 'value': 'd091e91b-ff5e-11e...","Ependymoma, NOS",4188.0,9391/3,Unknown,G2,Not Reported,C34440,d08d8323-ff5e-11e9-9a07-0a80fada099c
96,d09210a8-ff5e-11e9-9a07-0a80fada099c,"[{'system': 'PDC', 'value': 'd09210a8-ff5e-11e...","Glioma, NOS",3917.0,9400/3,Unknown,G1,Not Reported,C38991,d08d93b7-ff5e-11e9-9a07-0a80fada099c
97,db947c5a-b0c8-5e53-93f1-d24953c8bd0a,"[{'system': 'GDC', 'value': 'db947c5a-b0c8-5e5...",Glioblastoma,15964.0,9440/3,None,not reported,None,TCGA-02-2483,a2ac9937-f351-4d78-9261-264bf6c21e0c
98,e23338b6-3e43-577d-92a5-a4561610b305,"[{'system': 'GDC', 'value': 'e23338b6-3e43-577...","Oligodendroglioma, anaplastic",8752.0,9451/3,None,not reported,None,TCGA-DU-6408,14375753-a69f-41e0-b910-2bb8e84cc43a



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>primary_diagnosis:</b> The diagnosis instance that qualified a subject for inclusion on a ResearchProject.</li>
  <li><b>age_at_diagnosis:</b> The age in days of the individual at the time of diagnosis.</li>
  <li><b>morphology:</b> Code that represents the histology of the disease using the third edition of the <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a>, published in 2000, used principally in tumor and cancer registries for coding the site (topography) and the histology (morphology) of neoplasms.</li>
  <li><b>stage:</b> The extent of a cancer in the body. Staging is usually based on the size of the tumor, whether lymph nodes contain cancer, and whether the cancer has spread from the original site to other parts of the body. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> The degree of abnormality of cancer cells, a measure of differentiation, the extent to which cancer cells are similar in appearance and function to healthy cells of the same tissue type. The degree of differentiation often relates to the clinical behavior of the particular tumor. Based on the microscopic findings, tumor grade is commonly described by one of four degrees of severity. Histopathologic grade of a tumor may be used to plan treatment and estimate the future course, outcome, and overall prognosis of disease. Certain types of cancers, such as soft tissue sarcoma, primary brain tumors, lymphomas, and breast have special grading systems.</li>
  <li><b>method_of_diagnosis:</b> The method used to confirm the patients malignant diagnosis.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on research subjects that have a given diagnosis that meets our search criteria:

In [17]:
treatmentresults = myquery.treatment.run()
treatmentresults #2386

Getting results from database

Total execution time: 3801 ms



            QueryID: 6956c77e-9853-4adc-afc7-9a8c848df311
            
            Offset: 0
            Count: 100
            Total Row Count: 2386
            More pages: True
            

In [11]:

treatmentresults.to_dataframe()

Getting results from database

Total execution time: 3768 ms


,id,identifier,treatment_type,treatment_outcome,days_to_treatment_start,days_to_treatment_end,therapeutic_agent,treatment_anatomic_site,treatment_effect,treatment_end_reason,number_of_cycles,subject_id,researchsubject_id,researchsubject_diagnosis_id
0,01e3b1d2-99f0-5d32-9d1b-594429011f50,"[{'system': 'GDC', 'value': '01e3b1d2-99f0-5d3...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-FG-A70Z,98e82e98-341a-4bca-92c0-23c65c1bf94f,6db74e47-a996-5463-853a-6416b46a263a
1,08b045d6-5d7c-5ae5-94ba-3e07a239f555,"[{'system': 'GDC', 'value': '08b045d6-5d7c-5ae...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-DU-A6S3,1e713e76-064d-46dc-a53c-20a17eb68341,a8065153-05ac-5138-91ff-cfd23a8a3459
2,104f8ff0-48af-554a-9bfe-e5ed6b275bc9,"[{'system': 'GDC', 'value': '104f8ff0-48af-554...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-12-0821,eae44c1c-1628-4b58-8b90-d3372e3577d5,01af5bbe-5ce1-563c-934e-daa5124fb775
3,18c95fab-389e-5c64-949c-fbd2dd0bb2fe,"[{'system': 'GDC', 'value': '18c95fab-389e-5c6...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-08-0350,dc624b20-2854-4235-ae7e-672156ee30e3,1b04b930-35a8-514d-8a99-8bac2010dc21
4,26a729a6-dcf9-526d-a977-8bd24f17f5bf,"[{'system': 'GDC', 'value': '26a729a6-dcf9-526...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-DU-A7TI,0efaf8f9-177c-4ffb-b03f-f0316adf5dc1,472558de-4cbd-52cc-b03e-3e80f8275eee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,d194224d-2bfe-5fcf-a053-4ebe00b0c65a,"[{'system': 'GDC', 'value': 'd194224d-2bfe-5fc...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-FG-5963,68f550ca-3a33-45e4-b1c9-14f99e543263,df157cdd-174a-539c-ae8f-698a3b3a46a2
96,e6ffa328-c1be-5943-aae7-d9f0b741ac3e,"[{'system': 'GDC', 'value': 'e6ffa328-c1be-594...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-S9-A6WI,0b283f95-dbd4-4c36-9201-2198a90ee0d6,63c1ba76-12ba-5d0c-86ed-dccdfec708e0
97,f05078ba-26c4-5c33-830d-e44d95132310,"[{'system': 'GDC', 'value': 'f05078ba-26c4-5c3...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-S9-A6WO,a2202233-4a83-4163-9532-b6edb9941453,6b6695a0-d3ef-56a1-b980-d98871774b00
98,f85946e1-2a17-50d4-ad4d-d0b6f0b98b02,"[{'system': 'GDC', 'value': 'f85946e1-2a17-50d...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-76-6661,19dd2c8f-05ca-44e8-8c19-bac0327f2ea9,eed788c8-1c8c-59e0-a038-c9cb74800396



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i> Medication administration or other treatment types. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>treatment_type:</b> The treatment type including medication/therapeutics or other procedures.</li>
  <li><b>treatment_outcome:</b> The final outcome of the treatment.</li>
  <li><b>days_to_treatment_start:</b> The timepoint at which the treatment started.</li>
  <li><b>days_to_treatment_end:</b>The timepoint at which the treatment ended. </li>
  <li><b>therapeutic_agent:</b> One or more therapeutic agents as part of this treatment.</li>
  <li><b>treatment_anatomic_site:</b> The anatomical site that the treatment targets.</li>
  <li><b>treatment_effect:</b>The effect of a treatment on the diagnosis or tumor. </li>
  <li><b>treatment_end_reason:</b>The reason the treatment ended. </li>
  <li><b>number_of_cycles:</b>The number of treatment cycles the subject received. </li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id:</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---




### specimen

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [12]:
specimenresults =  myquery.specimen.run()
print(specimenresults) #39201

Getting results from database

Total execution time: 3966 ms

            QueryID: 663f60e8-5922-4872-9adc-64dbc0a1cac8
            
            Offset: 0
            Count: 100
            Total Row Count: 39201
            More pages: True
            


Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as seperate speciments. Let's look at a few:

In [13]:
specimenresults.to_dataframe()

,id,identifier,associated_project,days_to_collection,primary_disease_type,anatomical_site,source_material_type,specimen_type,derived_from_specimen,subject_id,researchsubject_id
0,008a69a1-9590-43a4-b5fd-14ab663721d5,"[{'system': 'GDC', 'value': '008a69a1-9590-43a...",TCGA-GBM,NaN,Gliomas,None,Solid Tissue Normal,aliquot,4a7395a9-b4b3-438f-afc7-25c634ba5c75,TCGA-08-0355,bb8230b5-f984-4d28-9f10-b3c89bab1c28
1,00c7f311-41b3-474e-91d2-0cc234846336,"[{'system': 'GDC', 'value': '00c7f311-41b3-474...",TCGA-LGG,NaN,Gliomas,None,Primary Tumor,aliquot,4cc6f946-09a0-403e-85c0-01f01c208113,TCGA-CS-6669,dc5bdd21-b23f-400e-bbe2-4c98a517fc9e
2,00ce2c37-cc9f-4868-8c9d-a76b0741651a,"[{'system': 'GDC', 'value': '00ce2c37-cc9f-486...",TCGA-GBM,NaN,Gliomas,None,Primary Tumor,aliquot,33d17296-27ea-4e0f-86e5-88c14d2775a3,TCGA-02-0086,8e3fbea7-d22e-40fc-8031-58fe6fe091bb
3,01729f63-439c-43c1-8ee9-943f059c542c,"[{'system': 'GDC', 'value': '01729f63-439c-43c...",TCGA-GBM,NaN,Gliomas,None,Primary Tumor,aliquot,8d859a3c-c509-41df-a485-185746721d21,TCGA-12-1598,2b90648c-c99e-4d4d-8d9b-67ff2c4e6115
4,01cee30c-fa9a-4ba8-b810-204bd1327004,"[{'system': 'GDC', 'value': '01cee30c-fa9a-4ba...",TCGA-GBM,NaN,Gliomas,None,Blood Derived Normal,aliquot,63251572-4138-4664-85bd-2939293c1f5e,TCGA-06-0190,74139255-a635-4c87-814d-3dd04ed630a8
...,...,...,...,...,...,...,...,...,...,...,...
95,38f4e4b7-a4cc-4d87-bfd0-c4356d92086e,"[{'system': 'GDC', 'value': '38f4e4b7-a4cc-4d8...",TCGA-LGG,NaN,Gliomas,None,Primary Tumor,aliquot,261e8179-6213-45d4-b132-3e39ea8b86fb,TCGA-HT-7467,f4a1fc14-0d32-4e77-a8a3-fd8fea82188a
96,3e9f8f4b-bd8b-4b46-9ed6-b95518ab80ad,"[{'system': 'GDC', 'value': '3e9f8f4b-bd8b-4b4...",TCGA-GBM,NaN,Gliomas,None,Primary Tumor,aliquot,cf7f3be6-3666-41a9-93a5-45c415087b98,TCGA-87-5896,ed879063-e603-4151-bf5e-22a4ee210281
97,3f1d6889-997a-4f7c-b74b-3def8db6db57,"[{'system': 'GDC', 'value': '3f1d6889-997a-4f7...",TCGA-GBM,NaN,Gliomas,None,Primary Tumor,aliquot,2239ec04-c589-448f-817b-7ef99aa65846,TCGA-02-0048,24eaab84-5d9c-4c12-af8b-32f4c6d8f81e
98,3faaa60f-7fe5-4b3c-acf2-2915b6f177c4,"[{'system': 'GDC', 'value': '3faaa60f-7fe5-4b3...",TCGA-LGG,0.0,Gliomas,None,Primary Tumor,slide,2fff9e3d-6b6c-566e-8a9d-f44647d76e4e,TCGA-P5-A731,335bc811-8bdc-4f66-bdd0-703b7cc7464b



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i><i>Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</i>
 A given specimen will have only a single subject ID and a single research subject ID</i>
    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>associated_project:</b> The Project associated with the specimen.</li>
  <li><b>days_to_collection:</b> The number of days from the index date to either the date a sample was collected for a specific study or project, or the date a patient underwent a procedure (e.g. surgical resection) yielding a sample that was eventually used for research.</li>
  <li><b>primary_disease_type:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
  <li><b>anatomical_site:</b> Per GDC Dictionary, the text term that represents the name of the primary disease site of the submitted tumor sample; recommend dropping tumor; biospecimen_anatomic_site.</li>
  <li><b>source_material_type:</b> The general kind of material from which the specimen was derived, indicating the physical nature of the source material.</li>
  <li><b>specimen_type:</b> The high-level type of the specimen, based on its how it has been derived from the original extracted sample. One of: analyte, aliquot, portion, sample, or slide.</li>
  <li><b>derived_from_specimen:</b> A source/parent specimen from which this one was directly derived.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, regardless of whether they are attached to subjects, research-subjects or specimens: 

In [14]:
myquery.file.run() #4549472

Getting results from database

Total execution time: 4183 ms



            QueryID: adbd32d8-8bfb-4934-ab74-582c91fa80bd
            
            Offset: 0
            Count: 100
            Total Row Count: 4549472
            More pages: True
            

In [15]:
fileresults = myquery.file.run()
fileresults.to_dataframe()

Getting results from database

Total execution time: 4187 ms


,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,imaging_series,researchsubject_specimen_id,researchsubject_id,subject_id
0,5e9a14f5-c5d8-4249-a7c6-5ab779544f83,"[{'system': 'GDC', 'value': '5e9a14f5-c5d8-424...",50e0e4de-6ee2-4fa8-b7fe-5bb3947c563c.wxs.VarSc...,Simple Nucleotide Variation,Annotated Somatic Mutation,MAF,TCGA-LGG,drs://dg.4DFC:5e9a14f5-c5d8-4249-a7c6-5ab77954...,162650,e0edbdec5cabeeabc08f38cbc99f32a2,Genomic,None,None,None,,,TCGA-DU-A7T6
1,345b67b8-ec76-4cd3-9f26-25aa9e0c9c19,"[{'system': 'GDC', 'value': '345b67b8-ec76-4cd...",TCGA-06-0881-01A-01-BS1.6f2cf2cc-dc8c-49ca-b36...,Biospecimen,Slide Image,SVS,TCGA-GBM,drs://dg.4DFC:345b67b8-ec76-4cd3-9f26-25aa9e0c...,260020245,ce066d859c4c7d452e10ea048dc340af,Genomic,None,None,None,,,TCGA-06-0881
2,027511f7-15e6-11ea-9bfa-0a42f3c845fe,"[{'system': 'PDC', 'value': '027511f7-15e6-11e...",01CPTAC_GBM_W_PNNL_20190123_B1S1_f24.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:027511f7-15e6-11ea-9bfa-0a42f3c8...,708638166,6e0470755a0ec6866519fd3a637f7cff,Proteomic,None,None,None,,,C3L-01834
3,3eb648c4-51d5-49e1-8c8e-596325c7deba,"[{'system': 'GDC', 'value': '3eb648c4-51d5-49e...",972ce08b-3d70-4e63-a572-b722905d6b4c.rna_seq.s...,Transcriptome Profiling,Splice Junction Quantification,TSV,TCGA-LGG,drs://dg.4DFC:3eb648c4-51d5-49e1-8c8e-596325c7...,2229707,e2a4392ebdde3fbbfe79eff034de9031,Genomic,None,None,None,,,TCGA-DB-A64X
4,b82d3c39-39e6-4a1d-a5f5-1ffe4c3d452e,"[{'system': 'PDC', 'value': 'b82d3c39-39e6-4a1...",NCI-11Plex-19-F11-f10554.mzML.gz,Processed Mass Spectra,Open Standard,mzML,Proteogenomic Analysis of Pediatric Brain Canc...,drs://dg.4DFC:b82d3c39-39e6-4a1d-a5f5-1ffe4c3d...,169024251,9b2c7a26ea571db3f992fba668f2e306,Proteomic,None,None,None,,,C907125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,a7590da5-eb4a-488a-bea0-6369a2167384,"[{'system': 'GDC', 'value': 'a7590da5-eb4a-488...",ffec4a68-74c4-4d7e-bc42-c60aebba3881_noid_Red....,DNA Methylation,Masked Intensities,IDAT,TCGA-LGG,drs://dg.4DFC:a7590da5-eb4a-488a-bea0-6369a216...,8095293,e083ac98e5a6573458ec797ea0eab229,Genomic,None,None,None,,,TCGA-HT-8109
96,0192d6c7-fb9b-4cc7-936a-4e7c9a766f7b,"[{'system': 'PDC', 'value': '0192d6c7-fb9b-4cc...",NCI-11Plex-9-F3-z11239.mzML.gz,Processed Mass Spectra,Open Standard,mzML,Proteogenomic Analysis of Pediatric Brain Canc...,drs://dg.4DFC:0192d6c7-fb9b-4cc7-936a-4e7c9a76...,238729165,6a62ad00d9368abf036a22723a0581bd,Proteomic,None,None,None,,,C25830
97,447da7db-c642-455a-980c-2350d7360e42,"[{'system': 'PDC', 'value': '447da7db-c642-455...",10CPTAC_GBM_W_PNNL_20190501_B3S2_f22.mzML.gz,Processed Mass Spectra,Open Standard,mzML,CPTAC3-Discovery,drs://dg.4DFC:447da7db-c642-455a-980c-2350d736...,191567974,92f7b3817f3b58ec61041374abe35013,Proteomic,None,None,None,,,C3L-02900
98,c1a2c48b-db96-4fb6-afeb-ee4846498cea,"[{'system': 'GDC', 'value': 'c1a2c48b-db96-4fb...",57b638b7-09bf-4607-8d9b-9bd59694aa4e_noid_Grn....,DNA Methylation,Masked Intensities,IDAT,CPTAC-3,drs://dg.4DFC:c1a2c48b-db96-4fb6-afeb-ee484649...,13676217,6dadcd6c38dd0e1a2f8b762a2ff2f663,Genomic,None,phs001287,None,,,C3N-02256


As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to chain your queries together, which we cover in the next tutorial <a href="../Chaining">Chaining Endpoints</a>: Combine information from multiple endpoints, and build And/Or/Like and other advanced query strings.

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the <a href="../DataSummaries">Data Summaries tutorial</a> to try it.



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>label:</b> Short name or abbreviation for dataset. Maps to rdfs:label.</li>
  <li><b>data_catagory:</b> Broad categorization of the contents of the data file.</li>
  <li><b>data_type:</b> Specific content type of the data file.</li>
  <li><b>file_format:</b> Format of the data files.</li>
  <li><b>associated_project:</b> A reference to the Project(s) of which this ResearchSubject is a member. The associated_project may be embedded using the ref definition or may be a reference to the id for the Project - or a URI expressed as a string to an existing entity.</li>
  <li><b>drs_uri:</b> A string of characters used to identify a resource on the Data Repo Service(DRS). Can be used to retreive this specific file from a server.</li>
  <li><b>byte_size:</b> Size of the file in bytes. Maps to dcat:byteSize.</li>
  <li><b>checksum:</b> The md5 value for the file. A digit representing the sum of the correct digits in a piece of stored or transmitted digital data, against which later comparisons can be made to detect errors in the data.</li>
  <li><b>data_modality:</b> Data modality describes the biological nature of the information gathered as the result of an Activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging".</li>
  <li><b>imaging_modality:</b> An imaging modality describes the imaging equipment and/or method used to acquire certain structural or functional information about the body. These include but are not limited to computed tomography (CT) and magnetic resonance imaging (MRI). Taken from the DICOM standard.</li>
  <li><b>dbgap_accession_number:</b> The dbgap accession number for the project.</li>
</ul>  

</div>
    
---